<a href="https://colab.research.google.com/github/Hkherdekar/Covid19/blob/master/Covid19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install apache-airflow

In [0]:
#importing libraries
import pandas as pd #Data processing
import numpy as np #Matrix/Linear algebraic operations
import random as rd #Random number 
from datetime import datetime
from datetime import date, timedelta #date formats
import pytz

#Visualization packages
from matplotlib import pyplot as plt
import seaborn as sns

#Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

from pandas import Series

#import all the required statistics functionality
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf

In [0]:
pst = pytz.timezone('America/Los_Angeles')

In [0]:
path_to_directory = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/'

In [21]:
#importing data (only last updated daily data file) and converting to pandas dataframe

date = datetime.now(pst) - timedelta(days=1)
date = date.strftime('%m-%d-%Y')
url = path_to_directory + date + '.csv'
CovidToday = pd.read_csv(url);


04-26-2020


In [22]:
CovidToday.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
0,45001.0,Abbeville,South Carolina,US,2020-04-27 02:30:52,34.223334,-82.461707,27,0,0,27,"Abbeville, South Carolina, US"
1,22001.0,Acadia,Louisiana,US,2020-04-27 02:30:52,30.295065,-92.414197,130,7,0,123,"Acadia, Louisiana, US"
2,51001.0,Accomack,Virginia,US,2020-04-27 02:30:52,37.767072,-75.632346,187,3,0,184,"Accomack, Virginia, US"
3,16001.0,Ada,Idaho,US,2020-04-27 02:30:52,43.452658,-116.241552,650,15,0,635,"Ada, Idaho, US"
4,19001.0,Adair,Iowa,US,2020-04-27 02:30:52,41.330756,-94.471059,1,0,0,1,"Adair, Iowa, US"


In [0]:
#importing timeseries data and converting to pandas dataframe
#The file links are hardcoded as they are auto-appended to the same file location

CovidCasesUS = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')

CovidCasesGlobal = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')

CovidDeathsUS = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')

CovidDeathsGlobal = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

CovidRecoveredGlobal = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')



```
All data imported
```

